![](https://news.dna3.com.mx/wp-content/uploads/2020/12/cenace-logo.png)
# TCN model train

En esta libreta se entrenará el modelo de TCN para la zona de carga HMO

In [0]:
#Instalamos las librerias necesarias
!pip install keras-tcn

     |▏                               | 3.6

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 4.9 MB 69.0 MB/s 
     |████████████████████████████████| 781 kB 76.6 MB/s 
     |████████████████████████████████| 232 kB 77.6 MB/s 
     |████████████████████████████████| 177 kB 76.8 MB/s 
     |████████████████████████████████| 155 kB 83.3 MB/s 
     |████████████████████████████████| 77 kB 4.2 MB/s 
     |████████████████████████████████| 151 kB 82.8 MB/s 
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Not uninstalling markupsafe at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-73281047-73f8-4f4d-9de1-5cd8dfefaf1e
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.5
    Not uninstalling protobuf at /databricks/python3/lib/python3.9/site

In [0]:
%pip install mlflow

Python interpreter will be restarted.
  Created wheel for databricks-cli: filename=databricks_cli-0.17.4-py3-none-any.whl size=142894 sha256=3ec7fa8d4f0556e1a92c0548da991524292297ee072334b53ea86f00d6c42a24
  Stored in directory: /home/spark-73281047-73f8-4f4d-9de1-5c/.cache/pip/wheels/b3/d4/24/ed7f421c3c600f3eee1a3a9e5c4aecc286a7622cac031918b7
Successfully built databricks-cli
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-73281047-73f8-4f4d-9de1-5cd8dfefaf1e
    Can't uninstall 'Jinja2'. No files were found to uninstall.
Python interpreter will be restarted.


### Importando bibliotecas necesarias

In [0]:
from pprint import pformat

import pandas as pd
import numpy as np
import json
import pickle 
from datetime import timedelta

import plotly.express as px
import plotly.graph_objects as go

import os
import joblib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score


import tensorflow
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tcn import TCN


### Data Reading and Processing

In [0]:
df =  pd.read_csv('https://raw.githubusercontent.com/carlosvelv/temp/main/DatasetHMO_01012016_29092022.csv')
df.head()
#Data cleaning
df.drop_duplicates(subset = ['Date'], inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.query('Date < 20220801')
df = df[df.Demand != 0].copy()
df.set_index('Date', inplace = True)
df = df.asfreq('H', method='pad')
df['Mes'] = df.index.month
df['Semana'] = df.index.weekofyear
df['Dia'] = df.index.weekday
df['Hora'] = df.index.hour
df.reset_index( inplace=True)

#Columnas a usar
#cols = ['Date','Demand', 'Temperature', 'Semana', 'Dia', 'Hora']
cols = ['Date', 'Demand']
df = df[cols].copy()
df.fillna(df.mean(), inplace=True)
df.set_index('Date', inplace=True)
df.head()

<command-1277713011816045>:11: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['Semana'] = df.index.weekofyear
<command-1277713011816045>:20: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df.fillna(df.mean(), inplace=True)


,Demand
Date,
2016-01-02 00:00:00,225.622
2016-01-02 01:00:00,213.620
2016-01-02 02:00:00,207.360
2016-01-02 03:00:00,201.374
2016-01-02 04:00:00,199.018


In [0]:
df_train = df[df.index.year < 2022]
df_test = df[df.index.year == 2022]

print(f"shape of df_train: {df_train.shape}\nshape of df_test: {df_test.shape}")

shape of df_train: (52584, 1)
shape of df_test: (5088, 1)


In [0]:
selected_attr = ['Demand'] 
train = df_train[selected_attr].copy()


In [0]:
def divide_series(series, n_pasado, n_futuro, n_salto, es_train=True):
    
    """
      series: ndarray con shape (instancias por hora, atributos en el orden establecido)
      n_pasado: número de observaciones pasadas para el encoder 
      n_futuro: número de observaciones futuras
      n_salto: a partir de donde empiezan a contar las observaciones futuras
      es_train: booleano, para generar una serie cada hora si es para aprendizaje.
                de lo contrario solo genera cada n_futuro (para prueba o validación)

    """
    X, y = list(), list() # Vamos a crear listas y al final hacemos ndarrays
    generador = range(series.shape[0]) if es_train else range(0, series.shape[0], n_futuro)

    for ini in generador:
        fin_anterior = ini + n_pasado
        fin_actual = fin_anterior + n_salto + n_futuro
        if fin_actual > len(series):
            break
        pasado = series[ini: fin_anterior, :]
        futuro = series[fin_anterior + n_salto: fin_actual, 0].reshape(-1, 1)
        X.append(pasado)
        y.append(futuro)
    return np.array(X), np.array(y)

In [0]:
def modelo_nn(n_pasado, n_attr, n_futuro):
    """
    Definiendo el modelo en forma funcional
    """
    ##
    model = Sequential([
    TCN(input_shape=(n_pasado, n_attr),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(n_futuro, activation='linear')
    ])
    return model


In [0]:
#Fixed params
selected_attr = ['Demand'] 
n_pasado = 24 * 7 + 12
n_futuro = 24
n_salto = 12
n_attr = len(selected_attr)
epochs = 10
opt = keras.optimizers.Adam()

In [0]:
### Generando serie de entrenamiento ###
X_train, y_train = divide_series(train.values, n_pasado, n_futuro, n_salto)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_attr))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))

In [0]:
modelo = modelo_nn(n_pasado, n_attr, n_futuro)

In [0]:
modelo.compile(
        optimizer=opt, 
        loss="mae"
    )

In [0]:
history = modelo.fit(
        X_train,
        y_train,
        epochs=epochs,
        validation_split=0.2,
        batch_size=32
    )

Epoch 1/10
 174/1310 [==>...........................] - ETA: 1:25 - loss: 347.7428

*** WARNING: max output size exceeded, skipping output. ***

1310/1310 [==============================] - 104s 80ms/step - loss: 37.3387 - val_loss: 36.3322


In [0]:
test = df_test[selected_attr].copy()

X_test, y_test = divide_series(test.values, n_pasado, n_futuro, n_salto, es_train=False)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_attr))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], 1))

# Calculando predicciones de prueba
y_est = modelo.predict(X_test)

7/7 [==============================] - 1s 28ms/step


In [0]:
y_est = modelo.predict(X_test)

7/7 [==============================] - 0s 25ms/step


In [0]:
y_test.shape

Out[23]: (204, 24, 1)

In [0]:
actual = y_test.ravel().reshape(-1, 1)
pred = y_est.ravel().reshape(-1, 1)

In [0]:
ape = np.abs((actual - pred) / actual)
mape = np.mean(ape) * 100
mape

Out[27]: 6.41344025446797

In [0]:
y_est.shape

Out[24]: (204, 24)

In [0]:
df_est = pd.DataFrame({
        "Real": actual.ravel(),
        "Estimado": pred.ravel(),
        "Fecha": test.index[n_pasado + n_salto:]     
    })

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_est.Fecha, y=df_est.Estimado, name="Estimada"))
fig.add_trace(go.Scatter(x=df_est.Fecha, y=df_est.Real, name="Real"))
fig.update_layout(title="Estimación de la demanda")